# GPT2 Masked Multi-head Self-attention详解

该实验可进行在线体验，在线体验链接（https://pangu.huaweicloud.com/gallery/asset-detail.html?id=6253fbfb-afe6-4727-bca5-5fc726541ab2
）

## 环境配置
>
>此为在线运行平台配置python3.9的指南，如在其他环境平台运行案例，请根据实际情况修改如下代码
>

1. 配置python3.9环境

In [ ]:
%%capture captured_output
!/home/ma-user/anaconda3/bin/conda create -n python-3.9.0 python=3.9.0 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.9.0/bin/pip install ipykernel

In [ ]:
import json
import os

data = {
   "display_name": "python-3.9.0",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.9.0/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.9.0/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

***注：以上代码执行完成后，需点击左上角或右上角将kernel更换为python-3.9.0***

<div align=center><img src="https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/imgs/ai-gallery/change-kernel.PNG"></div>

2. 安装mindspore2.2.12、indNLP及相关依赖，MindNLP官方仓详见：MindNLP

In [ ]:
%%capture captured_output
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.14/MindSpore/unified/x86_64/mindspore-2.2.14-cp39-cp39-linux_x86_64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install tokenizers==0.15.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
!wget https://repo.mindspore.cn/mindspore-lab/mindnlp/daily/202402/20240229/master_20240229160016_c5444092d8cfe47d73e292f25d9a9a56fb04828a_newest/any/mindnlp-0.2.0.20240229-py3-none-any.whl
!pip install mindnlp

***注：执行如上命令完成安装后，请点击上方的restart kernel图标重启kernel，再进行实验***

In [ ]:
# code from mindnlp and huggingface transformers

In [ ]:
import numpy as np
import mindspore
from mindspore import nn, ops, Tensor

## GPT-2 Self-attention: 1- Creating queries, keys, and values

![gpt2-self-attention-3.png](https://jalammar.github.io/images/gpt2/gpt2-self-attention-3.png)

In [ ]:
batch_size = 1
seq_len = 10
embed_dim = 768

# input x: (1, 10, 768)
x = Tensor(np.random.randn(batch_size, seq_len, embed_dim), mindspore.float32)

In [ ]:
from mindnlp._legacy.functional import split
from mindnlp.transformers.ms_utils import Conv1D

# query = Wq * X, key = Wk * X, value = Wv * X
# c_attn: (1, 10, 768*3) --> query, key, value: (1, 10, 768), (1, 10, 768), (1, 10, 768)
c_attn = Conv1D(3 * embed_dim, embed_dim)
query, key, value = split(c_attn(x), embed_dim, axis=2)
query.shape, key.shape, value.shape

![gpt2-self-attention-split-attention-heads-1.png](https://jalammar.github.io/images/gpt2/gpt2-self-attention-split-attention-heads-1.png)

![gpt2-self-attention-split-attention-heads-2.png](https://jalammar.github.io/images/gpt2/gpt2-self-attention-split-attention-heads-2.png)

In [ ]:
def split_heads(tensor, num_heads, attn_head_size):
    """
    Splits hidden_size dim into attn_head_size and num_heads
    """
    # (batch_size, seq_len, hidden_size) --> (batch_size, seq_len, num_heads, attn_head_size)
    new_shape = tensor.shape[:-1] + (num_heads, attn_head_size)
    tensor = tensor.view(new_shape)
    # (batch_size, seq_len, num_heads, attn_head_size) --> (batch_size, num_heads, seq_len, attn_head_size)
    return ops.transpose(tensor, (0, 2, 1, 3))

In [ ]:
num_heads = 12
head_dim = embed_dim // num_heads

# (1, 10, 768) --> (1, 10, 12, 64) --> (1, 12, 10, 64)
query = split_heads(query, num_heads, head_dim)
key = split_heads(key, num_heads, head_dim)
value = split_heads(value, num_heads, head_dim)

query.shape, key.shape, value.shape

## GPT-2 Self-attention: 2- Scoring

![gpt2-self-attention-scoring.png](https://jalammar.github.io/images/gpt2/gpt2-self-attention-scoring.png)

![](https://jalammar.github.io/images/gpt2/gpt2-self-attention-scoring-2.png)

In [ ]:
# qk点积
# q: (1, 12, 10, 64), k^T: (1, 12, 64, 10)
# attn_weights: (1, 12, 10, 10)
attn_weights = ops.matmul(query, key.swapaxes(-1, -2))

attn_weights.shape

![](https://jalammar.github.io/images/gpt2/transformer-decoder-attention-mask-dataset.png)

In [ ]:
# diagonal matrix to implement masked multi-head attention
# To ensure not to attend to future information
max_positions = seq_len

bias = Tensor(np.tril(np.ones((max_positions, max_positions))).reshape(
              (1, 1, max_positions, max_positions)), mindspore.bool_)
bias

![](https://jalammar.github.io/images/gpt2/queries-keys-attention-mask.png)

![](https://jalammar.github.io/images/gpt2/transformer-attention-mask.png)

In [ ]:
from mindnlp._legacy.functional import where, softmax

attn_weights = attn_weights / ops.sqrt(ops.scalar_to_tensor(value.shape[-1]))
query_length, key_length = query.shape[-2], key.shape[-2]
causal_mask = bias[:, :, key_length - query_length: key_length, :key_length].bool()
mask_value = Tensor(np.finfo(np.float32).min, dtype=attn_weights.dtype)
attn_weights = where(causal_mask, attn_weights, mask_value)

In [ ]:
np.finfo(np.float32).min

In [ ]:
attn_weights[0, 0]

![](https://jalammar.github.io/images/gpt2/transformer-attention-masked-scores-softmax.png)

In [ ]:
attn_weights = softmax(attn_weights, axis=-1)
attn_weights.shape

In [ ]:
attn_weights[0, 0]

![](https://jalammar.github.io/images/gpt2/gpt2-self-attention-multihead-sum-1.png)

In [ ]:
attn_output = ops.matmul(attn_weights, value)

attn_output.shape

## GPT-2 Self-attention: 3.5- Merge attention heads

![](https://jalammar.github.io/images/gpt2/gpt2-self-attention-merge-heads-1.png)

In [ ]:
def merge_heads(tensor, num_heads, attn_head_size):
    """
    Merges attn_head_size dim and num_attn_heads dim into hidden_size
    """
    # (batch_size, num_heads, seq_len, attn_head_size) --> (batch_size, seq_len, num_heads, seq_len)
    tensor = ops.transpose(tensor, (0, 2, 1, 3))
    new_shape = tensor.shape[:-2] + (num_heads * attn_head_size,)
    return tensor.view(new_shape)

In [ ]:
# (1, 12, 10, 64) --> (1, 10, 12, 64) --> (1, 10, 768)
attn_output = merge_heads(attn_output, num_heads, head_dim)

attn_output.shape

## GPT-2 Self-attention: 4- Projecting

![](https://jalammar.github.io/images/gpt2/gpt2-self-attention-project-1.png)

In [ ]:
c_proj = Conv1D(embed_dim, embed_dim)

In [ ]:
attn_output = c_proj(attn_output)
attn_output.shape